In [1]:
import os 
os.chdir ("/workspace/objectdet")

In [2]:
from mmengine.config import  Config

cfg = Config.fromfile('mmyolo/configs/yolov7/yolov7_l_syncbn_fast_8x16b-300e_coco.py')

In [3]:
cfg.model.bbox_head.head_module.num_classes = 1
cfg.model.bbox_head.loss_cls.loss_weight = cfg.loss_cls_weight * (1 / 80 * 3 / cfg.num_det_layers)
metainfo = dict(
    classes = ('Box',),
    palette = [
        (220, 20, 60),
    ]
)

cfg.train_dataloader = dict(
    batch_size=2,
    num_workers=2,
    persistent_workers=cfg.persistent_workers,
    pin_memory=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    collate_fn=dict(type='yolov5_collate'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root='datasets/Box-1/',
        ann_file='train/_annotations.coco.json',
        metainfo=metainfo,
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=False, min_size=32),
        pipeline=cfg.train_pipeline))

cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=1,
    persistent_workers=cfg.persistent_workers,
    pin_memory=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root='datasets/Box-1/',
        metainfo=metainfo,
        test_mode=True,
        data_prefix=dict(img='valid/'),
        ann_file='valid/_annotations.coco.json',
        pipeline=cfg.test_pipeline,
        batch_shapes_cfg=cfg.batch_shapes_cfg))

cfg.test_dataloader = dict(
    batch_size=1,
    num_workers=1,
    persistent_workers=cfg.persistent_workers,
    pin_memory=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root='datasets/Box-1/',
        metainfo=metainfo,
        test_mode=True,
        data_prefix=dict(img='test/'),
        ann_file='test/_annotations.coco.json',
        pipeline=cfg.test_pipeline,
        batch_shapes_cfg=cfg.batch_shapes_cfg))

cfg.val_evaluator = dict(
    type='mmdet.CocoMetric',
    proposal_nums=(100, 1, 10),
    ann_file='datasets/Box-1/' + 'valid/_annotations.coco.json',
    metric='bbox')

cfg.test_evaluator = dict(
    type='mmdet.CocoMetric',
    proposal_nums=(100, 1, 10),
    ann_file='datasets/Box-1/' + 'test/_annotations.coco.json',
    metric='bbox')

cfg.default_hooks = dict(
    param_scheduler=dict(
        type='YOLOv5ParamSchedulerHook',
        scheduler_type='linear',
        lr_factor=cfg.lr_factor,
        max_epochs=90),
    checkpoint=dict(
        type='CheckpointHook',
        interval=cfg.save_epoch_intervals,
        save_best='auto',
        max_keep_ckpts=cfg.max_keep_ckpts))

cfg.custom_hooks = [
    dict(
        type='EMAHook',
        ema_type='ExpMomentumEMA',
        momentum=0.0001,
        update_buffers=True,
        strict_load=False,
        priority=49),
]

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    clip_grad=dict(max_norm=10.0),
    optimizer=dict(
        type='SGD',
        lr=cfg.base_lr,
        momentum=0.937,
        weight_decay=cfg.weight_decay,
        nesterov=True,
        batch_size_per_gpu=8),
    constructor='YOLOv5OptimizerConstructor')

cfg.train_cfg = dict(
    type='EpochBasedTrainLoop',
    max_epochs=90,
    val_interval=1,
    dynamic_intervals=[(90 - cfg.num_epoch_stage2, cfg.val_interval_stage2)])

cfg.work_dir = 'work_dirs/yolov7_l_syncbn_fast_1x8b-90e_boxes/1'

In [4]:
cfg.dump('yolov7_l_syncbn_fast_1x8b-90e_boxes.py')

In [5]:
from mmengine.runner import Runner

model = Runner.from_cfg(cfg)
model.train()

03/16 15:39:51 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "log_processor" registry tree. As a workaround, the current "log_processor" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmyolo" is a correct scope, or whether the registry is initialized.
03/16 15:39:52 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 2066260759
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math

YOLODetector(
  (data_preprocessor): YOLOv5DetDataPreprocessor()
  (backbone): YOLOv7Backbone(
    (stem): Sequential(
      (0): ConvModule(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): SiLU(inplace=True)
      )
      (1): ConvModule(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): SiLU(inplace=True)
      )
      (2): ConvModule(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): SiLU(inplace=True)
      )
    )
    (stage1): Sequential(
      (0): ConvModule(
        (conv): Conv2d(64, 128, kernel_size=(3, 3)